In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from IPython.display import Javascript, display, clear_output, HTML

display(HTML("<style>.container { width:100% !important;}</style>"))

import pandas as pd

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [3]:
%run data_loading.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from typing import Literal

In [5]:
import numpy as np

In [6]:
fixtures_df = get_fixtures_df()
teams_df = get_teams_df()

In [7]:
cd ~/Documents/GitHub/predict_the_prem/

/Users/thomasrichardson/Documents/GitHub/predict_the_prem


In [8]:
from predict_the_prem.munge import ( 
    get_entries_position_df,
    get_entry_score_vs_game_week,
    get_league_table_df,
    get_team_fixtures_df,
    get_team_position_vs_game_week,
    merge_fixtures_and_teams,
)

In [9]:
fixtures_df = merge_fixtures_and_teams(fixtures_df=fixtures_df, teams_df=teams_df)

In [10]:
team_fixtures_df = get_team_fixtures_df(fixtures_df=fixtures_df)

In [11]:
league_table_df = get_league_table_df(team_fixtures_df=team_fixtures_df)

In [12]:
team_position_vs_game_week = get_team_position_vs_game_week(team_fixtures_df=team_fixtures_df)

In [13]:
entries = get_entries()

In [14]:
entries_position_df = get_entries_position_df(teams_df=teams_df, entries=entries)

In [15]:
get_entry_score_vs_game_week_df = get_entry_score_vs_game_week(team_fixtures_df, entries_position_df)

In [16]:
fixtures_df.head().T

,0,1,2,3,4
global_match_id,2444470,2444473,2444471,2444472,2444474
match_id,1,4,2,3,5
game_week,1,1,1,1,1
is_finished,True,True,True,True,True
is_finished_provisional,True,True,True,True,True
match_start_time,2024-08-16 19:00:00+00:00,2024-08-17 11:30:00+00:00,2024-08-17 14:00:00+00:00,2024-08-17 14:00:00+00:00,2024-08-17 14:00:00+00:00
is_provisional_start_time,False,False,False,False,False
is_started,True,True,True,True,True
home_team_id,14,10,1,8,15
away_team_id,9,12,20,5,17


In [81]:
def _expand_stats(stats):
    stats = eval(stats)
    stats = pd.DataFrame.from_records(stats)
    row_data = {}
    for idx, row in stats.iterrows():
        identifier = row['identifier']
        row_data[f'home_team_{identifier}_stats'] = row['h']
        row_data[f'away_team_{identifier}_stats'] = row['a']
    stats = pd.DataFrame([row_data])
    return row_data

In [82]:
test = pd.DataFrame.from_records(fixtures_df["stats"].apply(_expand_stats))

In [88]:
test[["home_team_goals_scored_stats", "away_team_goals_scored_stats"]].head().T

,0,1,2,3,4
home_team_goals_scored_stats,"[{'value': 1, 'element': 389}]",[],"[{'value': 1, 'element': 4}, {'value': 1, 'ele...",[],"[{'value': 1, 'element': 403}]"
away_team_goals_scored_stats,[],"[{'value': 1, 'element': 317}, {'value': 1, 'e...",[],"[{'value': 1, 'element': 113}, {'value': 1, 'e...",[]


In [16]:
bootstrap_url = os.path.join(base_url, "bootstrap-static/")
bootstrap_response = requests.get(bootstrap_url)

In [17]:
pd.DataFrame(bootstrap_response.json()["elements"]).sort_values("selected_rank_type")[["web_name", "selected_rank_type"]].head(20)

,web_name,selected_rank_type
619,Wan-Bissaka,1
139,Kelleher,1
385,M.Salah,1
248,João Pedro,1
0,Raya,2
505,Isak,2
234,Palmer,2
374,Frimpong,2
32,Gyökeres,3
386,Wirtz,3
